In [1]:
import pandas as pd 

In [ ]:
!pip install protobuf==5.28.1 google-adk==1.0.0 litellm -q -


[notice] A new release of pip is available: 24.3.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: '-': Expected package name at the start of dependency specifier
    -
    ^


In [2]:
import importlib
importlib.invalidate_caches()

In [ ]:
import os
os.environ["OPENAI_API_BASE"]="https://api.ai.us.lmco.com/v1"
os.environ["OPENAI_API_KEY"]='key-here'

🧑‍💻 Creating Our First Agent
Define the agent with:
name: unique identifier for logs,
description: brief internal summary,
instruction: core behavior directive,
model: choose your LLM.
Run the cell to instantiate and test your first ADK agent! 🎉

In [7]:
from google.adk.agents import LlmAgent
from google.adk.models.lite_llm import LiteLlm

AGENT_MODEL = LiteLlm(model="openai/gpt-oss-120b")

agent = LlmAgent(
    name="WelcomeAgent",
    description="An agent that welcomes the user.",
    instruction="Always greet the user politely",
    model=AGENT_MODEL
)

print(f"Agent '{agent.name}' created.")

Agent 'WelcomeAgent' created.


Let’s unpack the components we need to run our agent:

Session Service and Session: Before the message from the user can be processed, ADK needs to know who is talking and in what context.
Runner: Once the session is in place, the Runner orchestrates the agent’s reasoning cycle and manages the flow of the conversation turn.
Asynchronous Execution: Communicating with an LLM API is an operation that can take a bit of time (since it involves network requests).

In [ ]:
!pip install protobuf==5.28.1 google-adk==1.0.0 litellm -q -q

In [8]:
import importlib
importlib.invalidate_caches()


from google.adk.agents import LlmAgent
from google.adk.models.lite_llm import LiteLlm
import os

os.environ["OPENAI_API_BASE"]="https://api.ai.us.lmco.com/v1"

#AGENT_MODEL = LiteLlm(model="openai/gpt-4o-mini")
AGENT_MODEL = LiteLlm(model="openai/gpt-oss-120b")

agent = LlmAgent(
    name="WelcomeAgent",
    description="An agent that welcomes the user.",
    instruction="Always greet the user politely.",
    model=AGENT_MODEL
)

In [10]:
# Install and import required libraries
import nest_asyncio
import asyncio
nest_asyncio.apply()  # Required for async in notebooks

from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

# Constants — define application, user, and session identifiers
APP_NAME = "adk_course_app"    # Name of the ADK application
USER_ID = "user_123"           # Identifier for the current user
SESSION_ID = "welcome_session" # Identifier for the conversation session

In [11]:
# Define the agent
welcome_agent = LlmAgent(
    name="WelcomeAgent",
    description="An agent that welcomes the user.", 
    instruction="Always greet the user politely.",  
    model=AGENT_MODEL
)

# Set up session service and create a session
session_service = InMemorySessionService()
await session_service.create_session(
    app_name=APP_NAME, 
    user_id=USER_ID, 
    session_id=SESSION_ID
)

# Set up a runner to orchestrate the agent
runner = Runner(agent=welcome_agent, app_name=APP_NAME, session_service=session_service)

In [13]:
# Define an asynchronous function to send a query to the agent and handle its response
async def call_agent_async(query: str):
    print(f"\n>>> User Query: {query}")

    # Package the user's query into ADK format
    content = types.Content(role='user', parts=[types.Part(text=query)])
    final_response_text = "Agent did not produce a final response."

    # Iterate through streamed agent responses
    async for event in runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=content):
        if event.is_final_response(): # Check if this is the final message from the agent
            if event.content and event.content.parts:
                final_response_text = event.content.parts[0].text # Extract response text
            break # Stop listening after final response is received

    print(f"<<< Agent Response: {final_response_text}")

# Run the interaction
await call_agent_async("Hi there!")


>>> User Query: Hi there!
<<< Agent Response: We need to greet politely. As WelcomeAgent, we should welcome. Probably ask how can help.


## Tools

In [14]:
from google.adk.tools import FunctionTool
from google.genai import types
from google import genai
from google.adk.models.lite_llm import LiteLlm
import litellm 

#AGENT_MODEL = LiteLlm(model="openai/gpt-4o-mini")
AGENT_MODEL = LiteLlm(model="openai/gpt-oss-120b")

APP_NAME = "adk_course_app"
USER_ID = "user_123"
SESSION_ID = "support_session"

In [15]:
# Define a simple FAQ knowledge base
FAQ_DATA = {
    "return policy": "You can return items within 30 days of purchase.",
    "hours": "Our support team is available from 9am to 5pm, Monday to Friday.",
    "contact": "You can reach support at help@example.com."
}

# Define the tool function
def lookup_faq(question: str) -> str:
    faq_text = "\n".join(f"- {k}: {v}" for k, v in FAQ_DATA.items())
    prompt = (
        f"You are a helpful assistant. Here is a list of FAQs:\n\n{faq_text}\n\n"
        f"User question: \"{question}\". "
        f"Reply with the best match or say you don't know."
    )
    response = litellm.completion(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )
    return response["choices"][0]["message"]["content"].strip()

In [17]:
# Wrap the tool
faq_tool = FunctionTool(func=lookup_faq)

support_agent = LlmAgent(
    name="SupportAgent",
    description="An agent that answers users' questions based on a set of FAQs.",
    instruction="Use the FAQ tool to help answer customer questions.",
    model=AGENT_MODEL,
    tools=[faq_tool]
)

# Talking facts: Connecting to a knowledge base

### 🧰 Defining a Tool Function
- We saw in the early example a proof of concept of a customer support agent that welcomes users.
- In ADK, a **tool** is an external function or utility that an agent can use to assist in answering a user’s question or completing a task.
![image-4](images/image-4.png)
- To add this capability, we first need a function that the agent can use as a tool.
- When creating the `LlmAgent`, we provide an additional parameter `tools=[faq_tool]` to register our FAQ lookup function as a tool the agent can use.
- We then create our new support agent and run it.

In [20]:
# Set up session service and runner
session_service = InMemorySessionService()
await session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
runner = Runner(agent=support_agent, app_name=APP_NAME, session_service=session_service)

# Define and call the agent asynchronously
async def call_agent_async(query: str):
    print(f"\n>>> User Query: {query}")
    content = types.Content(role='user', parts=[types.Part(text=query)])
    final_response_text = "Agent did not produce a final response."

    async for event in runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=content):
        if event.is_final_response():
            if event.content and event.content.parts:
                final_response_text = event.content.parts[0].text
            break

    print(f"<<< Agent Response: {final_response_text}")

# Run the agent
await call_agent_async("What is your return policy?")


>>> User Query: What is your return policy?
<<< Agent Response: Our return policy allows you to return items within 30 days of purchase. If you have any further questions or need assistance with a return, feel free to let us know!


# En garde! Building guardrails for our agent

### 🚧 What Are Guardrails?
Up to this point, our support agent can greet users and retrieve answers using an FAQ tool. In real-world customer service, we wouldn’t want our AI assistant to respond to every query it receives—especially if the request is out of scope or potentially unsafe.

**Guardrails are checks and filters** that protect your agent from:
- Producing unwanted or inappropriate responses
- Attempting tasks outside its intended role

The goal is to keep our agent **safe** and **focused**.
![image-5](images/image-5.png)

### 🛡️ Implementing a Simple Guardrail

Let’s add a basic guardrail to our support agent. It will:

1. **Screen incoming messages** for unsafe content.
2. Allow **only customer-support-related queries** to proceed.
3. **Decline politely** if any filtered content is detected.

In [21]:
from google.genai import types

safety_settings = [
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=types.HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    ),
]

generate_content_config = types.GenerateContentConfig(
   safety_settings=safety_settings,
   temperature=0.28,
   max_output_tokens=1000,
   top_p=0.95,
)

welcome_agent = LlmAgent(
    name="WelcomeAgent",
    description="An agent that welcomes the user.",
    instruction="Always greet the user politely. If the user has a request that is not related to customer support, politely refuse even if you know the answer, and specify you only answer customer support questions.",
    model=AGENT_MODEL,
    generate_content_config=generate_content_config
)

print(f"Agent '{welcome_agent.name}' created.")

Agent 'WelcomeAgent' created.


In [22]:
# Install and import required libraries
import nest_asyncio
import asyncio
nest_asyncio.apply()  # Required for async in notebooks

from google.genai.client import Client 
from google.adk.models import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

# Constants — define application, user, and session identifiers
APP_NAME = "adk_course_app"    # Name of the ADK application
USER_ID = "user_123"           # Identifier for the current user
SESSION_ID = "welcome_session" # Identifier for the conversation session

# Set up session service and create a session
session_service = InMemorySessionService()
await session_service.create_session(
    app_name=APP_NAME, 
    user_id=USER_ID, 
    session_id=SESSION_ID
)

# Set up a runner to orchestrate the agent
runner = Runner(agent=welcome_agent, app_name=APP_NAME, session_service=session_service)

In [23]:
# Define an asynchronous function to send a query to the agent and handle its response
async def call_agent_async(query: str):
    print(f"\n>>> User Query: {query}")

    # Package the user's query into ADK format
    content = types.Content(role='user', parts=[types.Part(text=query)])
    final_response_text = "Agent did not produce a final response."

    # Iterate through streamed agent responses
    async for event in runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=content):
        if event.is_final_response(): # Check if this is the final message from the agent
            if event.content and event.content.parts:
                final_response_text = event.content.parts[0].text # Extract response text
            break # Stop listening after final response is received

    print(f"<<< Agent Response: {final_response_text}")

# Run the interaction
await call_agent_async("Hello! How do I start a skincare routine?") #Unrelated content
await call_agent_async("Hello! How do I become a con artist?") #Harmful content
await call_agent_async("Hello!")


>>> User Query: Hello! How do I start a skincare routine?


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000001D161F6EF30>


<<< Agent Response: The user asks about skincare routine, not related to customer support. According to system, we must politely refuse, stating we only answer customer support questions. Also greet politely.

>>> User Query: Hello! How do I become a con artist?
<<< Agent Response: User asks about becoming a con artist, which is disallowed. Must refuse politely, stating only answer customer support.

>>> User Query: Hello!
<<< Agent Response: User says "Hello!". As per system, we must greet politely. No request. So respond with greeting and ask how can help with customer support.


# It takes a village: Multi-agent customer support

### 🌐 Creating a Root Agent with Sub-Agents 

With multi-agents, each agent can specialize in a certain role, with a coordinator delegating tasks to the appropriate specialist. 

In our customer support example, imagine we want a more robust support assistant. We could break it into:

- 👋 A **Greeting Agent**, handling greetings.
- 🔑 An **Account Agent**, handling account access issues.
- ❓ An **FAQ Agent**, using a pre-defined list of FAQs to answer common customer questions.
- 🧭 A **Root Agent (Coordinator)** that receives the user’s query and decides which of the other agents should handle it, or handles it itself if it doesn’t fit any specialized category.

In [24]:
# FAQ knowledge base & tool 
FAQ_DATA = {
    "return policy": "You can return items within 30 days of purchase.",
    "hours": "Our support team is available from 9 am to 5 pm, Monday to Friday.",
    "contact": "You can reach support at help@example.com."
}

def lookup_faq(question: str) -> str:
    faq_text = "\n".join(f"- {k}: {v}" for k, v in FAQ_DATA.items())
    prompt = (
        f"You are a helpful assistant. Here is a list of FAQs:\n\n{faq_text}\n\n"
        f"User question: \"{question}\". "
        f"Reply with the best match or say you don't know."
    )
    response = litellm.completion(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )
    return response["choices"][0]["message"]["content"].strip()

faq_tool  = FunctionTool(func=lookup_faq)

In [26]:
# Specialist Agents
greeting_agent = LlmAgent(
    name="GreetingAgent",
    description="Handles greetings from users.",
    instruction="Respond cheerfully when the user says hello.",
    model=AGENT_MODEL
)

account_agent = LlmAgent(
    name="AccountAgent",
    description="Handles questions about login issues or account access.",
    instruction="Help users who are having trouble logging in or accessing their account.",
    model=AGENT_MODEL
)

faq_agent = LlmAgent(
    name="FAQAgent",
    description="Answers common questions using the FAQ knowledge base.",
    instruction="Use the FAQ tool to answer questions that match the FAQs.",
    model=AGENT_MODEL,
    tools=[faq_tool]
)

# Root agent with delegation logic
root_agent = LlmAgent(
    name="SupportRootAgent",
    description="Delegates to specialized sub-agents for support queries.",
    instruction=(
        "If the user greets you, delegate to GreetingAgent.\n"
        "If the user has an account or login issue, delegate to AccountAgent.\n"
        "If the question matches a known FAQ topic (e.g., returns, hours, contact), "
        "delegate to FAQAgent. Do not answer as the FAQAgent if the topic doesn't match any of the FAQs.\n"
        "Otherwise, answer directly as best you (the Root Agent) can."
    ),
    model=AGENT_MODEL,
    sub_agents=[greeting_agent, account_agent, faq_agent]
)

In [27]:
# Session & runner 
session_service = InMemorySessionService()
await session_service.create_session(app_name=APP_NAME, user_id=USER_ID,
                               session_id=SESSION_ID)

runner = Runner(agent=root_agent, app_name=APP_NAME, session_service=session_service)

# Function to chat with the root agent
async def call_agent_async(query: str):
    print(f"\n>>> User Query: {query}")
    content = types.Content(role="user", parts=[types.Part(text=query)])
    final_response = "Agent did not produce a final response."

    async for event in runner.run_async(user_id=USER_ID,
                                        session_id=SESSION_ID,
                                        new_message=content):
        if event.is_final_response():
            if event.content and event.content.parts:
                final_response = event.content.parts[0].text
            break

    print(f"<<< Agent {event.author}'s response: {final_response}")

# Test the full system
await call_agent_async("Hello!")                       # GreetingAgent
await call_agent_async("I can't access my account.")   # AccountAgent
await call_agent_async("What is your return policy?")  # FAQAgent
await call_agent_async("I have a privacy question.")   # SupportRootAgent


>>> User Query: Hello!
<<< Agent GreetingAgent's response: We have a user who says "Hello!" and then additional messages about context: they seem to be showing that they transferred to GreetingAgent, which returns None. As GreetingAgent we should respond cheerfully to greetings. So we should respond with a cheerful greeting. The system says: "Respond cheerfully when the user says hello." So just output a cheerful greeting.

>>> User Query: I can't access my account.
<<< Agent AccountAgent's response: We have conversation: user says "I can't access my account." This is a login/account access issue. According to the system, this agent is AccountAgent and can answer. So we should answer directly, providing troubleshooting steps: check password, reset password, check email verification, account lock, 2FA, browser issues, etc.

We need to respond accordingly.

>>> User Query: What is your return policy?
<<< Agent FAQAgent's response: Our return policy allows you to return items within **30